In [1]:
import pandas as pd
from haversine import haversine as hvs, Unit
import numpy as np
from collections import Counter
import math
from datetime import datetime, timedelta

# Procesar samples_with_nan

Crear una columna con un indice para los samples (1 sample es un recorrido completo)

In [ ]:
samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
# len(samples_with_nan['init_stop'].unique())   #83 paraderos  # 2560 samples con nan
#samples_with_nan['id_sample'] = samples_with_nan.apply(lambda row: int(row.name)//82) 
#samples_with_nan.apply(lambda row: int(row.name))#(//82) 
samples_with_nan['id_sample'] = samples_with_nan.index.map(lambda row: int(row)//83)
columna = samples_with_nan.pop('id_sample') # Extraemos la columna C y la guardamos en una variable
samples_with_nan.insert(0, 'id_sample', columna) # Insertamos la columna C en la posición 1
#samples_with_nan.to_csv('samples_with_nan.csv')
samples_with_nan.to_csv('samples_with_nan.csv', index=False)
samples_with_nan

Si no se borró la columna 'Unnamed: 0', eliminarla y volver a guardar el dataframe en csv

In [ ]:
#EJECUTAR CUANDO EXISTA LA COLUMNA UNNAMED 0
samples_with_nan = pd.read_csv('samples_with_nan.csv', index_col=None)
samples_with_nan.drop(columns=['Unnamed: 0'], inplace=True)
samples_with_nan.to_csv('samples_with_nan.csv', index=False)
samples_with_nan

### Filtro de samples de la columna time_travel para mostrar todos los menores a 12 segundos

In [5]:
samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
#samples_with_nan.nlargest(50,columns='time_travel')
#samples_with_nan[samples_with_nan['time_travel']>=960]['id_sample'].unique #960 segundos equivale a 10 minutos
samples_with_nan[samples_with_nan['time_travel']<12]['id_sample'].unique().shape #960 segundos equivale a 10 minutos

(13,)

In [11]:
samples_with_nan.nlargest(30, 'time_travel')

,id_sample,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
57676,694,109,110,109:110,2023-03-07 06:31:39,2023-03-07 20:23:56,49937.0,14166346.0,14311788.0,SJ18,5.0,2023-03-07
57675,694,108,109,108:109,2023-03-07 20:23:05,2023-03-07 06:31:39,49886.0,14311656.0,14166346.0,SJ18,5.0,2023-03-07
104579,1259,117,118,117:118,2023-03-13 16:45:39,2023-03-13 07:01:27,35052.0,15222719.0,15115550.0,SJ25,3.0,2023-03-13
159233,1918,74,75,74:75,2023-03-19 05:32:34,2023-03-19 14:49:02,33388.0,16167716.0,16225225.0,SJ38,3.0,2023-03-19
159261,1918,102,103,102:103,2023-03-19 06:02:57,2023-03-19 15:18:24,33327.0,16169211.0,16228375.0,SJ38,3.0,2023-03-19
159257,1918,98,99,98:99,2023-03-19 06:00:34,2023-03-19 15:15:46,33312.0,16169197.0,16228156.0,SJ38,3.0,2023-03-19
159232,1918,73,74,73:74,2023-03-19 14:47:26,2023-03-19 05:32:34,33292.0,16225047.0,16167716.0,SJ38,3.0,2023-03-19
159260,1918,101,102,101:102,2023-03-19 15:17:21,2023-03-19 06:02:57,33264.0,16228268.0,16169211.0,SJ38,3.0,2023-03-19
159256,1918,97,98,97:98,2023-03-19 15:14:25,2023-03-19 06:00:34,33231.0,16227926.0,16169197.0,SJ38,3.0,2023-03-19
191083,2302,52,53,52:53,2023-03-23 06:24:57,2023-03-23 12:42:09,22632.0,16838722.0,16914486.0,SJ06,3.0,2023-03-23


### Filtro de samples de la columna time_travel para moestrar todos los mayores a 960 segundos(10 minutos)

In [9]:
samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
#samples_with_nan.nlargest(50,columns='time_travel')
#samples_with_nan[samples_with_nan['time_travel']>=960]['id_sample'].unique #960 segundos equivale a 10 minutos
samples_with_nan[samples_with_nan['time_travel']>=960]['id_sample'].unique().shape #960 segundos equivale a 10 minutos

(56,)

In [12]:
samples_with_nan.nsmallest(30, 'time_travel')

,id_sample,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
167873,2022,82,83,82:83,2023-03-20 06:13:43,2023-03-20 06:13:53,10.0,16269633.0,16269656.0,SJ37,1.0,2023-03-20
1014,12,53,54,53:54,2023-03-01 10:55:49,2023-03-01 10:56:00,11.0,13233012.0,13233054.0,SJ05,2.0,2023-03-01
32070,386,67,68,67:68,2023-03-04 14:10:54,2023-03-04 14:11:05,11.0,13820185.0,13820219.0,SJ05,4.0,2023-03-04
46344,558,65,66,65:66,2023-03-06 12:54:52,2023-03-06 12:55:03,11.0,14059032.0,14059079.0,SJ05,3.0,2023-03-06
53899,649,67,68,67:68,2023-03-06 09:52:49,2023-03-06 09:53:00,11.0,14026068.0,14026111.0,SJ41,2.0,2023-03-06
117409,1414,82,83,82:83,2023-03-14 07:02:29,2023-03-14 07:02:40,11.0,15288247.0,15288284.0,SJ41,1.0,2023-03-14
129433,1559,71,72,71:72,2023-03-16 09:18:19,2023-03-16 09:18:30,11.0,15669142.0,15669187.0,SJ05,2.0,2023-03-16
146127,1760,82,83,82:83,2023-03-17 05:25:43,2023-03-17 05:25:54,11.0,15816980.0,15816990.0,SJ37,1.0,2023-03-17
147149,1772,108,109,108:109,2023-03-17 13:14:12,2023-03-17 13:14:23,11.0,15904671.0,15904705.0,SJ41,3.0,2023-03-17
147154,1772,113,114,113:114,2023-03-17 13:17:33,2023-03-17 13:17:44,11.0,15905251.0,15905286.0,SJ41,3.0,2023-03-17


# Porcentaje de nan en time_travel

In [6]:
samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
#samples_with_nan.nsmallest(50,columns='time_travel')
samples_with_nan['time_travel'].isna().sum()/samples_with_nan['time_travel'].shape[0]

0.14896460843373494

## Filtrar por fecha 

In [7]:
samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
samples_with_nan['date_time_init'] = pd.to_datetime(samples_with_nan['date_time_init'])
samples_with_nan['date_time_end'] = pd.to_datetime(samples_with_nan['date_time_end'])

fecha_filtro = pd.to_datetime('2023-03-12').date()
df_filtrado = samples_with_nan[samples_with_nan['date_time_init'].dt.date == fecha_filtro]
df_filtrado.nsmallest(20, ['time_travel'])

,id_sample,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
95540,1151,42,43,42:43,2023-03-12 11:55:46,2023-03-12 11:55:58,12.0,15047959.0,15047983.0,SJ04,1.0,2023-03-12
95541,1151,43,44,43:44,2023-03-12 11:55:58,2023-03-12 11:56:10,12.0,15047983.0,15048008.0,SJ04,1.0,2023-03-12
95553,1151,55,56,55:56,2023-03-12 12:10:40,2023-03-12 12:10:52,12.0,15049606.0,15049626.0,SJ04,1.0,2023-03-12
95581,1151,83,84,83:84,2023-03-12 12:30:36,2023-03-12 12:30:48,12.0,15052341.0,15052344.0,SJ04,1.0,2023-03-12
95604,1151,106,107,106:107,2023-03-12 12:57:31,2023-03-12 12:57:43,12.0,15055259.0,15055291.0,SJ04,1.0,2023-03-12
95606,1151,108,109,108:109,2023-03-12 12:57:59,2023-03-12 12:58:11,12.0,15055322.0,15055343.0,SJ04,1.0,2023-03-12
95611,1151,113,114,113:114,2023-03-12 13:01:13,2023-03-12 13:01:25,12.0,15055652.0,15055672.0,SJ04,1.0,2023-03-12
95613,1151,115,116,115:116,2023-03-12 13:02:27,2023-03-12 13:02:39,12.0,15055782.0,15055803.0,SJ04,1.0,2023-03-12
95614,1151,116,117,116:117,2023-03-12 13:02:39,2023-03-12 13:02:51,12.0,15055803.0,15055820.0,SJ04,1.0,2023-03-12
95626,1152,45,46,45:46,2023-03-12 06:28:34,2023-03-12 06:28:46,12.0,15013171.0,15013176.0,SJ05,1.0,2023-03-12


In [8]:
samples_with_nan = pd.read_csv('../data/samples_1v.csv', index_col=None)
samples_with_nan['date_time_init'] = pd.to_datetime(samples_with_nan['date_time_init'])
samples_with_nan['date_time_end'] = pd.to_datetime(samples_with_nan['date_time_end'])

fecha_filtro = pd.to_datetime('2023-03-12').date()
df_filtrado = samples_with_nan[samples_with_nan['date_time_init'].dt.date == fecha_filtro]
df_filtrado.nsmallest(20, ['time_travel'])

,number,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
98150,44,79,80,79:80,2023-03-12 11:49:25,2023-03-12 08:53:30,-10555.0,15047288.0,15023026.0,SJ31,1.0,2023-03-12
98182,76,111,112,111:112,2023-03-12 12:19:34,2023-03-12 09:24:00,-10534.0,15051151.0,15027886.0,SJ31,1.0,2023-03-12
98176,70,105,106,105:106,2023-03-12 12:16:32,2023-03-12 09:21:06,-10526.0,15050280.0,15027629.0,SJ31,1.0,2023-03-12
98162,56,91,92,91:92,2023-03-12 12:02:40,2023-03-12 09:09:26,-10394.0,15048763.0,15026206.0,SJ31,1.0,2023-03-12
98158,52,87,88,87:88,2023-03-12 11:02:03,2023-03-12 09:03:47,-7096.0,NaN,NaN,NaN,NaN,NaN
98157,51,86,87,86:87,2023-03-12 11:56:29,2023-03-12 11:02:03,-3266.0,NaN,NaN,NaN,NaN,NaN
96607,78,113,114,113:114,2023-03-12 11:13:07,2023-03-12 11:13:15,8.0,NaN,NaN,NaN,NaN,NaN
96537,8,43,44,43:44,2023-03-12 10:08:41,2023-03-12 10:08:51,10.0,NaN,NaN,NaN,NaN,NaN
96856,78,113,114,113:114,2023-03-12 14:02:36,2023-03-12 14:02:46,10.0,NaN,NaN,NaN,NaN,NaN
96268,71,106,107,106:107,2023-03-12 11:51:49,2023-03-12 11:52:00,11.0,NaN,NaN,NaN,NaN,NaN


## Calcular promedio

In [ ]:
# calcular el promedio
means = []
for col in df_sp.columns:
    means.append(round(df_sp[col].mean(), 0))

print(means)

In [ ]:
gg = pd.read_csv('samples.csv')
gg

In [ ]:
def curar(row, df):
    print(row.name)
    if math.isnan(row['time_travel']):
        row['time_travel'] = means[row.name]
        if row.name == 0:
            row['date_time_end'] =  df.loc[row.name + 1, 'date_time_init']
            row['date_time_init'] = row['date_time_end'] - datetime.timedelta(seconds= row['time_travel'])
        else:
            row['date_time_init'] =  df.loc[row.name - 1, 'date_time_end']
            row['date_time_end'] =  row['date_time_init'] + datetime.timedelta(seconds= row['time_travel'])
    return row
    

for i, row in gg.iterrows():
        if math.isnan(row['time_travel']):
            travel_t = means[row.name]
            if row.name == 0:
                datetime_end =  gg.loc[i + 1, 'date_time_init']
                datetime_init = datetime_end - datetime.timedelta(seconds=travel_t)
            else:
                datetime_init =  df_af.loc[i - 1, 'date_time_end']
                datetime_end =  datetime_init + datetime.timedelta(seconds=travel_t)
            df_af.loc[df_af.shape[0]] = [row['init_stop'], row['end_stop'], row['linkref'],datetime_init, datetime_end, travel_t, np.nan, np.nan]
        else:
            df_af.loc[df_af.shape[0]] = list(row)

In [ ]:
# list_samples[85]
df_tmp = pd.concat(list_samples)
df_tmp.to_csv('samples.csv', index=False)